# Flagging days/periods with Snow and Heavy Rain



In [3]:
# import relevant modules
import configparser
from psycopg2 import connect
import psycopg2.sql as pg
import pandas.io.sql as pandasql
import datetime

# connect to database
CONFIG = configparser.ConfigParser()
CONFIG.read(r'C:\Users\alouis2\Documents\Python Scripts\db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

The following code automates the flagged data creation process through two functions. `formatter` takes in a dictionary who's keys are snow/heavy rain dates and who's values are time periods with snow corresponding to their key. This function returns a string that is used by the `flagged` function that returns a dataframe containing flagged data. 

In [8]:
# run query
march = {datetime.date(2018,3,1):['Evening'], \
          datetime.date(2018,3,7):['Midday'],\
          datetime.date(2018,3,9):['Midday'],\
          datetime.date(2018,3,12):['AM Peak', 'Midday', 'PM Peak'], \
          datetime.date(2018,3,15):['Midday', 'PM Peak'], \
          datetime.date(2018,3,29):['Midday', 'PM Peak', 'Evening']}

def flagged(x):
    def formatter(x):
        string = ''' dt = '%s' '''
        for i in range(len(x)-1):
            string = string + '''or dt = '%s' '''
        string = string % tuple([key for key in x])
        return  '''select pilot.dt, pilot.street, pilot.direction, pilot.period, 
                    pilot.tt as pilot, baseline.tt as baseline 
                    from king_pilot.dash_daily_dev as pilot,  king_pilot.dash_baseline_dev as baseline 
                    where (%s)
                    and category = 'Pilot' and  pilot.day_type = 'Weekday' and pilot.street = baseline.street 
                    and pilot.direction = baseline.direction and pilot.day_type = baseline.day_type 
                    and pilot.period = baseline.period order by dt, street, direction, period''' % string
    string = formatter(x)
    sql = pg.SQL(string)
    snow = pandasql.read_sql(sql, con)
    change = [snow.pilot.values[i]-snow.baseline.values[i] for i in range(len(snow))]
    snow['change'] = change
    flag = [0 for i in range(len(snow))]
    snow['flag'] = flag 
    for key in x:
        for i in x[key]:
            for j in range(len(snow)):
                if snow.dt.values[j] == key and snow.period.values[j] == i:
                    snow.flag.values[j] += 1        
    return snow



In [9]:
flagged(march)

,dt,street,direction,period,pilot,baseline,change,flag
0,2018-03-01,Adelaide,Eastbound,AM Peak,7.794884,7.864729,-0.069845,0
1,2018-03-01,Adelaide,Eastbound,Evening,7.863057,8.071130,-0.208072,1
2,2018-03-01,Adelaide,Eastbound,Midday,8.067379,9.410475,-1.343095,0
3,2018-03-01,Adelaide,Eastbound,PM Peak,9.321537,14.699972,-5.378435,0
4,2018-03-01,Bathurst,Northbound,AM Peak,5.916050,5.298643,0.617407,0
5,2018-03-01,Bathurst,Northbound,Evening,5.427688,5.412162,0.015527,1
6,2018-03-01,Bathurst,Northbound,Midday,5.993247,5.607554,0.385694,0
7,2018-03-01,Bathurst,Northbound,PM Peak,6.267730,6.108102,0.159628,0
8,2018-03-01,Bathurst,Southbound,AM Peak,6.608192,5.109653,1.498539,0
9,2018-03-01,Bathurst,Southbound,Evening,5.099174,5.248978,-0.149805,1
